In [1]:
import pandas as pd
import pickle
from tqdm import tqdm
import os

import torch
from torch import nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import numpy as np
import random
import os
import glob

from utils import (InputExample, InputFeatures, _truncate_seq_pair, Args,
                      convert_examples_to_features, load_and_cache_examples, train, evaluate)

from pytorch_transformers import (
    WEIGHTS_NAME, BertConfig,
    BertForSequenceClassification,
    BertTokenizer
)

In [25]:
# All important paths and constants

small = False

folds_number = 10

args = Args()
aggregation_level = "Chapter"

main_dir = "/mnt/HDD/bportelli/lab_avanzato"

original_data_path = "/mnt/HDD/bportelli/lab_avanzato/beatrice.pkl"

diagnosis_df_preprocessed_serialized = main_dir+"/input_df_dropped{}.pkl".format("_small" if small else "")
models_path = main_dir+"/models/"
model_name = args.model_name_or_path + "_small" if small else args.model_name_or_path
model_directory = "{}{}/{}/".format(models_path, model_name, aggregation_level)
model_directory_estimators = "{}{}/{}/Estimators/".format(models_path, model_name, aggregation_level)
model_directory_training = "{}{}/{}/Training/".format(models_path, model_name, aggregation_level)
model_directory_predictions = "{}{}/{}/Predictions/".format(models_path, model_name, aggregation_level)

if not os.path.exists(models_path):
    os.mkdir(models_path)
if not os.path.exists(model_directory):
    os.makedirs(model_directory, exist_ok=True)
if not os.path.exists(model_directory_estimators):
    os.mkdir(model_directory_estimators)
if not os.path.exists(model_directory_training):
    os.mkdir(model_directory_training)
if not os.path.exists(model_directory_predictions):
    os.mkdir(model_directory_predictions)

all_features_path = model_directory+"all_features.pkl"
all_label_codes_path = model_directory+"all_label_codes.pkl"
label_map_path = model_directory+"label_map.pkl"
evaluation_path = model_directory+"evaluation_results.pkl"

print(models_path)
print(model_directory)
print(model_directory_estimators)
print(model_directory_training)
print(model_directory_predictions)

/mnt/HDD/bportelli/lab_avanzato/models/
/mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/
/mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/Estimators/
/mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/Training/
/mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/Predictions/


In [3]:
with open(original_data_path, "rb") as o:
    input_df = pickle.load(o)

print("Original dataframe")
input_df.head()

Original dataframe


,Text,Leaf,Category,Block,Chapter
0,"RIF. TRAUMA AL 3° DITO DELLA MANO SN, RISALENT...",9249,924,17.10,CH_17
1,RIFERISCE TRAUMA EMICOSTATO DX IN SEGUITO A PD...,78002,780,16.1,CH_16
2,RISCONRO DI HB 7.5 STENOSI PILORICA DI NDD,78900,789,16.1,CH_16
3,SPALLA SX TRAUMA CONTUSIVO EMICOSTATO SIN,9249,924,17.10,CH_17
4,DOLORE E BRUCIORE ALLA BOCCA DELLO STOMACO DA ...,07999,079,1.9,CH_1


In [4]:
# ## Let us see how diagnosis are distributed among each:
# - Leaf
# - Category
# - Block
# - Chapter

def compute_relative_frequency(dataframe, label):
    dataframe_freq = dataframe[label].value_counts().to_frame().reset_index().rename(columns={'index':label, label: 'Frequency-Absolute'})
    dataframe_freq["Frequency-Relative"] = dataframe_freq["Frequency-Absolute"].div(len(dataframe))
    dataframe_freq["Frequency-Relative(%)"] = dataframe_freq["Frequency-Absolute"].div(len(dataframe))*100
    return dataframe_freq

diagnosis_df_annotated = input_df

df_leaf_freq = compute_relative_frequency(diagnosis_df_annotated, "Leaf")
df_category_freq = compute_relative_frequency(diagnosis_df_annotated, "Category")
df_block_freq = compute_relative_frequency(diagnosis_df_annotated, "Block")
df_chapter_freq = compute_relative_frequency(diagnosis_df_annotated, "Chapter")

print("Relative/Absolute frequencies for Chapters")
df_chapter_freq

Relative/Absolute frequencies for Chapters


,Chapter,Frequency-Absolute,Frequency-Relative,Frequency-Relative(%)
0,CH_17,114509,0.273845,27.384474
1,CH_16,84664,0.202471,20.247134
2,CH_6,42548,0.101752,10.175223
3,CH_1,38956,0.093162,9.316207
4,CH_7,32222,0.077058,7.705792
5,CH_13,22862,0.054674,5.467377
6,CH_8,21481,0.051371,5.137115
7,CH_9,18301,0.043766,4.376628
8,CH_10,10955,0.026199,2.619854
9,CH_12,10382,0.024828,2.482823


In [5]:
if os.path.exists(diagnosis_df_preprocessed_serialized):
    diagnosis_df_annotated = pd.read_pickle(diagnosis_df_preprocessed_serialized)
    print("Length before: {}".format(len(diagnosis_df_annotated)))
    print("Dataframe preprocessed loaded from path: {}".format(diagnosis_df_preprocessed_serialized))
    print("Length after: {}".format(len(diagnosis_df_annotated)))
else:
    for index, row in tqdm(diagnosis_df_annotated.iterrows(), desc="Processed", total=len(diagnosis_df_annotated)):
            text = row["Text"]
            # PRELIMINARY OPERATIONS
            # Removing \n, \r and ° chars
            text = text.replace('\r', ' ')
            text = text.replace('\n', ' ')
            text = text.replace('°', ' ')
            # TOKENIZATION 
            # tokens = nlp(text)
            # PUNCTUATION REMOVAL
            # tokens = [token for token in tokens if token.pos_ != 'PUNCT']
            # LEMMATIZATION
            # lemmas = [t.lemma_ for t in tokens]
            # SAVING PRE PROCESSED TEXT
            # lemmatized_text = ' '.join(lemmas)
            lemmatized_text = text
            diagnosis_df_annotated.at[index,'Text-Processed'] = lemmatized_text

    df_frequency = compute_relative_frequency(diagnosis_df_annotated, aggregation_level)
    print("Filtering entries for a number of folds equal to: {}".format(folds_number))
    print("Length before: {}".format(len(diagnosis_df_annotated)))
    for index, row in df_frequency.iterrows():
        code = row[aggregation_level]
        frequency = row["Frequency-Absolute"]
        if frequency < folds_number:
            diagnosis_df_annotated.drop(diagnosis_df_annotated[diagnosis_df_annotated[aggregation_level] == code].index,inplace=True)
    
    if small:
        diagnosis_df_annotated = diagnosis_df_annotated.iloc[:10,]
    
    print("Length after: {}".format(len(diagnosis_df_annotated)))

    diagnosis_df_annotated.to_pickle(diagnosis_df_preprocessed_serialized)
    print("Dataframe preprocessed saved at path: {}".format(diagnosis_df_preprocessed_serialized))

diagnosis_df_annotated.head()

Length before: 10
Dataframe preprocessed loaded from path: /mnt/HDD/bportelli/lab_avanzato/input_df_dropped_small.pkl
Length after: 10


,Text,Leaf,Category,Block,Chapter,Text-Processed
0,"RIF. TRAUMA AL 3° DITO DELLA MANO SN, RISALENT...",9249,924,17.10,CH_17,"RIF. TRAUMA AL 3 DITO DELLA MANO SN, RISALENT..."
1,RIFERISCE TRAUMA EMICOSTATO DX IN SEGUITO A PD...,78002,780,16.1,CH_16,RIFERISCE TRAUMA EMICOSTATO DX IN SEGUITO A PD...
2,RISCONRO DI HB 7.5 STENOSI PILORICA DI NDD,78900,789,16.1,CH_16,RISCONRO DI HB 7.5 STENOSI PILORICA DI NDD
3,SPALLA SX TRAUMA CONTUSIVO EMICOSTATO SIN,9249,924,17.10,CH_17,SPALLA SX TRAUMA CONTUSIVO EMICOSTATO SIN
4,DOLORE E BRUCIORE ALLA BOCCA DELLO STOMACO DA ...,07999,079,1.9,CH_1,DOLORE E BRUCIORE ALLA BOCCA DELLO STOMACO DA ...


In [6]:
label_list = input_df[aggregation_level].unique().tolist()
num_labels = len(label_list)

config_class, model_class, tokenizer_class = BertConfig, BertForSequenceClassification, BertTokenizer
config = config_class.from_pretrained(args.config_name if args.config_name else args.model_name_or_path, num_labels=num_labels)
tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name if args.tokenizer_name else args.model_name_or_path, do_lower_case=args.do_lower_case)

if os.path.exists(all_features_path):
    with open(all_features_path, "rb") as o:
        features = pickle.load(o)
    print("All features loaded from path: {}".format(all_features_path))
else:
    text_list = diagnosis_df_annotated["Text-Processed"].values.tolist()

    all_examples = []
    for idx, text in tqdm(enumerate(text_list), desc="Creating examples", total=len(diagnosis_df_annotated)):
        example = InputExample(guid=idx, text_a=text, label=diagnosis_df_annotated.loc[idx][aggregation_level])
        all_examples.append(example)

    for i in range(3):
        print(all_examples[i])

    features = convert_examples_to_features(all_examples, label_list, args.max_seq_length, tokenizer,
                cls_token=tokenizer.cls_token,
                cls_token_segment_id=0,
                sep_token=tokenizer.sep_token,
                pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                pad_token_segment_id=0,
                args = args)
    with open(all_features_path, "wb") as o:
        pickle.dump(features, o)
    print("All features saved at path: {}".format(all_features_path))
    
for i in range(3):
    print(features[i])

All features loaded from path: /mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/all_features.pkl
InputFeatures("0",..., "0")
InputFeatures("1",..., "1")
InputFeatures("2",..., "1")


In [7]:
if os.path.exists(all_label_codes_path):
    with open(all_label_codes_path, "rb") as o:
        all_label_codes = pickle.load(o)
    with open(label_map_path, "rb") as o:
        label_map = pickle.load(o)
    print("All labels loaded from path: {}".format(all_label_codes_path))

else:
    all_labels = diagnosis_df_annotated[aggregation_level].to_list()
    label_map = {label : i for i, label in enumerate(label_list)}
    all_label_codes = [label_map[l] for l in all_labels]

    del all_labels
    
    with open(all_label_codes_path, "wb") as o:
        pickle.dump(all_label_codes, o)
    with open(label_map_path, "wb") as o:
        pickle.dump(label_map, o)
    print("All labels saved to path: {}".format(all_label_codes_path))
    
print(all_label_codes[0:5])

All labels loaded from path: /mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/all_label_codes.pkl
[0, 1, 1, 0, 2]


In [8]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if torch.cuda.is_available() and args.use_cuda :
        torch.cuda.manual_seed_all(args.seed)

device = torch.device("cuda" if torch.cuda.is_available() and args.use_cuda else "cpu")
args.device = device
set_seed(args)

model = model_class.from_pretrained(args.model_name_or_path, from_tf=bool('.ckpt' in args.model_name_or_path), config=config)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_af

In [9]:
print("Current model: {}".format(model_name))
fold_counter = 1

print(model_directory)
print(model_directory_estimators)
print(model_directory_training)

Current model: bert-base-multilingual-uncased_small
/mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/
/mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/Estimators/
/mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/Training/


In [10]:
from sklearn.model_selection import StratifiedKFold

codes = all_label_codes

data = {}

sk_fold = StratifiedKFold(n_splits=folds_number)

if args.do_train:

    for training_indexes, test_indexes in sk_fold.split(X=np.zeros(len(codes)), y=codes):
        print("Processing fold: {}".format(fold_counter))
        # Merging together rows of the CSR matrix selected to be the training set of the current fold
        training_data = []
        training_labels = []
        test_data = []
        test_labels = []
        for index in training_indexes:
            training_data.append(features[index])
            training_labels.append(all_label_codes[index])
        for index in test_indexes:
            test_data.append(features[index])
            test_labels.append(all_label_codes[index])

        # Convert to Tensors and build dataset
        all_unique_ids = torch.tensor([f.unique_id for f in training_data], dtype=torch.long) 
        all_input_ids = torch.tensor([f.input_ids for f in training_data], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in training_data], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in training_data], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in training_data], dtype=torch.long)

        train_dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids, all_unique_ids)

        train(args, train_dataset, model, tokenizer)

        # Some serialization and things

        data["Fold-{}".format(fold_counter)] = {}
        data["Fold-{}".format(fold_counter)]["Training-Data"] = training_data
        data["Fold-{}".format(fold_counter)]["Training-Labels"] = training_labels
        data["Fold-{}".format(fold_counter)]["Test-Data"] = test_data
        data["Fold-{}".format(fold_counter)]["Test-Labels"] = test_labels
        model_output_dir = "{}{}-Fold-{}-Model".format(model_directory_estimators, model_name, fold_counter)
        data_serialized = "{}{}-Fold-{}-Data.pkl".format(model_directory_training, model_name, fold_counter)

        if not os.path.exists(model_output_dir):
            os.makedirs(model_output_dir)

        model_to_save = model.module if hasattr(model, 'module') else model
        model_to_save.save_pretrained(model_output_dir)
        torch.save(args, os.path.join(model_output_dir, 'training_args.bin'))
        tokenizer.save_pretrained(model_output_dir)

        print("Model serialized at path: {}".format(model_output_dir))

        if not os.path.exists(data_serialized):
            with open(data_serialized, "wb") as output_file:
                pickle.dump(data, output_file)
                print("Training data serialized at path: {}".format(data_serialized))

        # The fold is processed        

        fold_counter+=1

    # The training is completed

    print("{} training done.".format(model_name))

In [11]:
if args.do_eval:

    for fold_counter in range(1, folds_number+1):
        print("Processing fold: {}".format(fold_counter))

        model_output_dir = "{}{}-Fold-{}-Model".format(model_directory_estimators, model_name, fold_counter)
        data_serialized = "{}{}-Fold-{}-Data.pkl".format(model_directory_training, model_name, fold_counter)
        prediction_serialized = "{}{}-Fold-{}-Prediction.pkl".format(model_directory_predictions, model_name, fold_counter)
        all_predictions_serialized = "{}{}-Fold-{}-All-Predictions.pkl".format(model_directory_predictions, model_name, fold_counter)

        model = model_class.from_pretrained(model_output_dir)
        tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
        model.to(args.device)

        with open(data_serialized, "rb") as input_file:
            training_data = pickle.load(input_file)

        test_data = training_data["Fold-{}".format(fold_counter)]["Test-Data"]

        # Actual prediction

        # Convert to Tensors and build dataset
        all_unique_ids = torch.tensor([f.unique_id for f in test_data], dtype=torch.long) 
        all_input_ids = torch.tensor([f.input_ids for f in test_data], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in test_data], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in test_data], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in test_data], dtype=torch.long)

        test_dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids, all_unique_ids)

        prediction, all_predictions = evaluate(args, test_dataset, model, tokenizer)

        # Some serialization and things

        with open(prediction_serialized, "wb") as output_file:
            pickle.dump(prediction, output_file)
            print("Prediction serialized at path: {}".format(prediction_serialized))
        with open(all_predictions_serialized, "wb") as output_file:
            pickle.dump(all_predictions, output_file)
            print("All-Predictions serialized at path: {}".format(prediction_serialized))

    # The predictions are completed

    print("{} predictions done.".format(model_name))

Processing fold: 1


Evaluating: 100%|██████████| 2/2 [00:00<00:00, 29.24it/s]

***** Running evaluation  *****
  Num examples = %d 7
  Batch size = %d 4
Prediction serialized at path: /mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/Predictions/bert-base-multilingual-uncased_small-Fold-1-Prediction.pkl
All-Predictions serialized at path: /mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/Predictions/bert-base-multilingual-uncased_small-Fold-1-Prediction.pkl
Processing fold: 2



Evaluating: 100%|██████████| 1/1 [00:00<00:00, 33.45it/s]

***** Running evaluation  *****
  Num examples = %d 3
  Batch size = %d 4
Prediction serialized at path: /mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/Predictions/bert-base-multilingual-uncased_small-Fold-2-Prediction.pkl
All-Predictions serialized at path: /mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/Predictions/bert-base-multilingual-uncased_small-Fold-2-Prediction.pkl
bert-base-multilingual-uncased_small predictions done.


In [27]:
def compute_accuracy(real_labels, best_preds, all_preds):
    acc_1 = (real_labels == best_preds).mean()
    acc_3 = np.mean([1 if r in a[:3] else 0 for (r,a) in zip(real_labels, all_preds)])
    acc_5 = np.mean([1 if r in a[:5] else 0 for (r,a) in zip(real_labels, all_preds)])
    return acc_1, acc_3, acc_5

results_df = pd.DataFrame(columns=["Fold", "Accuracy@1", "Accuracy@3", "Accuracy@5"])

for fold_counter in range(1, folds_number+1):
    print("Processing fold: {}".format(fold_counter))

    data_serialized = "{}{}-Fold-{}-Data.pkl".format(model_directory_training, model_name, fold_counter)
    prediction_serialized = "{}{}-Fold-{}-Prediction.pkl".format(model_directory_predictions, model_name, fold_counter)
    all_predictions_serialized = "{}{}-Fold-{}-All-Predictions.pkl".format(model_directory_predictions, model_name, fold_counter)

    with open(data_serialized, "rb") as input_file:
        data = pickle.load(input_file)
        
    with open(prediction_serialized, "rb") as input_file:
        best_prediction = pickle.load(input_file)
        
    with open(all_predictions_serialized, "rb") as input_file:
        all_predictions = pickle.load(input_file)
    
    real_labels = data["Fold-{}".format(fold_counter)]["Test-Labels"]
    
    acc_1, acc_3, acc_5 = compute_accuracy(real_labels, best_prediction, all_predictions)
    results_df = results_df.append({"Fold": fold_counter,
                       "Accuracy@1": acc_1,
                       "Accuracy@3": acc_3,
                       "Accuracy@5": acc_5}, ignore_index=True)

results_df.to_pickle(evaluation_path)
print("Evaluation results saved to path: {}".format(evaluation_path))

Processing fold: 1
Processing fold: 2
Evaluation results saved to path: /mnt/HDD/bportelli/lab_avanzato/models/bert-base-multilingual-uncased_small/Chapter/evaluation_results.pkl
